In [1]:
import scanpy as sc
import os
import math
import itertools
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
warnings.filterwarnings("ignore")
import seaborn as sns

In [2]:
# 设置
sc.settings.verbosity = 3             # 设置日志等级: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.6 anndata==0.9.2 umap==0.5.5 numpy==1.23.5 scipy==1.10.1 pandas==1.5.3 scikit-learn==1.3.2 statsmodels==0.14.0 igraph==0.10.8 louvain==0.8.1 pynndescent==0.5.11


In [ ]:
##T

In [3]:
adata = sc.read_h5ad("./clean0615.h5ad")

In [5]:
adata.obs.celltype_Major.unique()

['SMCs', 'Endothilial_PLVAP+VFM+', 'Fibroblasts', 'B/Plasma cells', 'Myeloid cells', 'Epithelial cells', 'T/NK cells', 'Mast cells', 'Endothilial_PLVAP+VFM-', 'Endothilial_PLVAP-']
Categories (10, object): ['B/Plasma cells', 'Endothilial_PLVAP+VFM+', 'Endothilial_PLVAP+VFM-', 'Endothilial_PLVAP-', ..., 'Mast cells', 'Myeloid cells', 'SMCs', 'T/NK cells']

In [6]:
adata_T = adata[(adata.obs['celltype_Major'].isin(["T/NK cells"]))]

In [7]:
adata_T.X = adata_T.raw.X.todense()

In [8]:
sc.pp.normalize_per_cell(adata_T, counts_per_cell_after=1e4)
sc.pp.log1p(adata_T)

normalizing by total count per cell
    finished (0:00:04): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


In [9]:
sc.pp.highly_variable_genes(adata_T,n_top_genes=2000)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:03)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [11]:
adata_T.var.highly_variable = adata_T.var.highly_variable & [ not x.startswith(('RPL', 'RPS', 'MT-', 'MRPS', 'MRPL')) for x in adata_T.var.index ]
sum(adata_T.var.highly_variable)

1986

In [12]:
sc.pp.scale( adata_T, )
sc.tl.pca( adata_T, svd_solver='arpack', use_highly_variable = True)

computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:03)


In [13]:
if True:
    sc.external.pp.harmony_integrate(adata_T, 'sample',  
                                     basis = 'X_pca',  
                                     adjusted_basis= 'X_pca_harmony', 
                                     )

2024-07-24 10:48:24,607 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2024-07-24 10:48:30,723 - harmonypy - INFO - sklearn.KMeans initialization complete.
2024-07-24 10:48:30,799 - harmonypy - INFO - Iteration 1 of 10
2024-07-24 10:48:33,936 - harmonypy - INFO - Iteration 2 of 10
2024-07-24 10:48:36,814 - harmonypy - INFO - Converged after 2 iterations


In [14]:
sc.pp.neighbors(adata_T, n_pcs=20,)

computing neighbors
    using 'X_pca' with n_pcs = 20
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:17)


In [15]:
sc.tl.umap(adata_T)

computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:21)


In [15]:
sc.tl.louvain(adata_T,resolution=1,key_added="cluter_cell_1")

running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 12 clusters and added
    'cluter_cell_1', the cluster labels (adata.obs, categorical) (0:00:00)


In [16]:
from statannot import add_stat_annotation


In [17]:
### myloiad  重复 T 细胞

In [18]:
###  cellponedb

In [20]:
"""
cellphonedb method statistical_analysis --counts-data ensembl --output-path output meta.txt counts.txt
cellphonedb plot heatmap_plot --pvalues-path pvalues.txt --output-path output --pvalue 0.05 --count-name heatmap_count.pdf --log-name test.heatmap_log_count.pdf --count-network-name count_network.txt --interaction-count-name interaction_count.txt meta.txt
"""

In [21]:
import os
import anndata as ad
import pandas as pd
import ktplotspy as kpy
import matplotlib.pyplot as plt
from plotnine import ggplot, aes, geom_point, theme,element_blank
from plotnine import *

In [22]:
adata = ad.read_h5ad("adata_allraw.h5ad")

# 2) output from CellPhoneDB
means = pd.read_csv("means.txt", sep="\t")
pvals = pd.read_csv("pvalues.txt", sep="\t")
decon = pd.read_csv("deconvoluted.txt", sep="\t")


In [ ]:
p = kpy.plot_cpdb(
    adata=adata,
    cell_type1="Fibroblasts",
    cell_type2="Endothelial_PLVAP+VFM+|Endothelial_PLVAP-|Epithelial cells|Myeloid cells|Endothelial_PLVAP+VFM-|",
    means=means[b],
    pvals=pvals[b],
    celltype_key="celltype_Major",
    highlight_size=1,
    genes=["SPP1","CD274","PDCD1","CCL3","NOTCH1","CXCL","CCL4"],
    figsize=(6.5, 10.5),
    title="Major Ligand-Receptor of HC",
#         default_style=False,

)
p= p+ theme(panel_grid_major = element_blank(), panel_grid_minor = element_blank(),
            figure_size=(5, 10),text=element_text(size=16),axis_text_x=element_text(rotation=45,hjust=1),
           axis_title=element_blank()
           
           )

In [26]:
p.save("F2D.CHC.pdf")

In [29]:
plt1 = kpy.plot_cpdb_heatmap(pvals=pvals, figsize=(8, 8), title="Sum of significant interactions",
                 
                     )
plt1.ax_heatmap.set_xticklabels(plt1.ax_heatmap.get_xmajorticklabels(), fontsize = 14)
plt1.ax_heatmap.set_yticklabels(plt1.ax_heatmap.get_xmajorticklabels(), fontsize = 14)
plt1.savefig("figures/F2A.pdf")

In [48]:
CHC = pd.read_csv("count_network.txt",sep="\t")

In [ ]:
key = "Fibroblasts"
Fibroblasts = CHC[CHC.SOURCE == key]
names = ['Endothilial_PLVAP+VFM-',
 'Endothilial_PLVAP+VFM+',
         'Endothilial_PLVAP-',
 'Myeloid cells',
 'T/NK cells',
 'B/Plasma cells',
 'SMCs',
 'Mast cells',
 'Fibroblasts',
 'Epithelial cells'
        ]

In [ ]:
cout = list(Fibroblasts.set_index("TARGET").T[names].T['count'])

In [ ]:
import matplotlib
cmap = matplotlib.cm.get_cmap("tab20")
listc = []
for i in range(30):
    listc.append(cmap(i))


In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
n_vertices = 10
edges = [(0, 8),(1, 8),(2, 8),(3,8),(4,8),(5,8),(6,8),(7,8),(8,8),(9,8)]
g = ig.Graph(n_vertices, edges)

# Set attributes for the graph, nodes, and edges
# g["title"] = "Small Social Network"
g.vs["name"] = names
# g.es["married"] = [False, False, False, False, False, False, False, True]

# Set individual attributes
# g.vs[1]["name"] = "Kathy Morillas"
# g.es[0]["married"] = True

# Plot in matplotlib
# Note that attributes can be set globally (e.g. vertex_size), or set individually using arrays (e.g. vertex_color)
fig, ax = plt.subplots(figsize=(7,7))
ig.plot(
    g,
    target=ax,
    layout="circle", # print nodes in a circular layout
    vertex_size=0.3,
    vertex_color=listc[0:10],
    vertex_frame_width=1.0,
    vertex_frame_color="white",
    vertex_label=g.vs["name"],
    vertex_label_size=12,
        edge_curved="0.1",
    edge_label=cout,edge_label_size=14,
    
    autocurve=True,
    edge_width=list(Fibroblasts['count']/60),
#     edge_curved=1,
    edge_color=listc[0:10],edge_arrow_size="12",
)

# plt.show()
plt.title("Sum of HC Fibroblasts")
plt.savefig("./figures/F2HC.F.pdf")

In [ ]:
###  trajectory

In [ ]:
"""
library(monocle)
library(ggplot2)
library(cowplot)
library(dplyr)
fd <- read.delim("genes.tsv", row.names = 1)
fd <- new("AnnotatedDataFrame", data = fd)
data <- read.table("RCexp.tsv",)
pd <- read.delim("meta.tsv", row.names = 1)
pd <- new("AnnotatedDataFrame", data = pd)

HSMM <- newCellDataSet(as.matrix(data), phenoData = pd, featureData = fd,)
HSMM <- estimateSizeFactors(HSMM)
HSMM <- estimateDispersions(HSMM)
HSMM <- detectGenes(HSMM, min_expr = 0.1)
print(head(fData(HSMM)))

expressed_genes <- row.names(subset(fData(HSMM), num_cells_expressed >= 10))
write.table(expressed_genes, file = paste("./", "/", "expressed_genes.tsv", sep = ""), sep = "\t", quote = F, row.names = F, col.names = T)
HSMM <- HSMM[expressed_genes]
x <- pData(HSMM)$num_genes_expressed
x_1 <- (x - mean(x)) / sd(x)
df <- data.frame(x = x_1)

clustering_DEG_genes <- dispersionTable(HSMM)
disp.genes <- subset(clustering_DEG_genes, mean_expression >= 0.1 & dispersion_empirical >= 1 * dispersion_fit)$gene_id
write.table(disp.genes, file = paste("./", "/", "disp_genes.tsv", sep = ""), sep = "\t", quote = F, row.names = F, col.names = T)
HSMM <- setOrderingFilter(HSMM, disp.genes)
HSMM_rd <- reduceDimension(HSMM, max_components = 2, num_dim = 5, method = 'DDRTree', verbose = TRUE,)
HSMM_sort <- orderCells(HSMM_rd,)
saveRDS(HSMM_sort, file = paste("./", "/", "CelldataSet.rds",sep = ""))

df <- data.frame(barcode = row.names(pData(HSMM_sort)), pData(HSMM_sort))
write.table(df, file = paste("./", "/", "Pseudotime.tsv", sep = ""), sep = "\t", quote = F, row.names = F, col.names = T)
"""
# R4.1

In [ ]:
"""
 library(monocle)
 library(ggplot2)
 library(cowplot)
 library(dplyr)
 HSMM_sort <- readRDS(paste("./",'/',"CelldataSet.rds",sep = ""))
 HSMM_sort$cell_typeA1 <- as.character(HSMM_sort$cell_typeA1)

 df <- data.frame(barcode = row.names(pData(HSMM_sort)), pData(HSMM_sort))
 pdf(file = paste("./",'/', "pseudotime.pdf",sep = ""), height = 5, width = 10)
 plot_cell_trajectory(HSMM_sort, color_by = "Pseudotime") #Pseudotime
 dev.off()
 pdf(file = paste("./",'/', "state.pdf",sep = ""), height =  5, width = 10)
 plot_cell_trajectory(HSMM_sort, color_by = "State") #Pseudotime
 dev.off()"""
